In [1]:
import json 
import string 
import random
import nltk 
import numpy as np
from nltk.stem import WordNetLemmatizer
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt")
nltk.download("wordnet")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kshit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kshit\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
file_path = 'intents_chatgpt.json'
with open(file_path, 'r') as file:
    data = json.load(file)

In [3]:
data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi',
    'Hey',
    'How are you',
    'Is anyone there?',
    'Hello',
    'Good day',
    'Whats up',
    'Whats cooking'],
   'responses': ['Hello!',
    'Good to see you again!',
    'Hi there, how can I help?',
    "Hello! I'm Dexter. How may I help you?",
    'Hey there!'],
   'context_set': ''},
  {'tag': 'book_table',
   'patterns': ['Book a table',
    'Can I book a table?',
    'I want to book a table',
    'Book seat',
    'I want to book a seat',
    'Can I book a seat?'],
   'responses': ["Sure, I can help you with that! Please provide me with the date and time you'd like to book."],
   'context_set': ''},
  {'tag': 'available_tables',
   'patterns': ['How many seats are available?',
    'Available seats',
    'How many tables are available?',
    'Available tables'],
   'responses': ['We currently have X tables/seats available.'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['cya',
    'See you later',
    '

In [4]:
words =[]
classes =[]
data_x =[]
data_y =[]

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        data_x.append(pattern)
        data_y.append(intent["tag"]),
        
    if intent["tag"] not in classes:
        classes.append(intent["tag"])

lemmatizer = WordNetLemmatizer()

words=[lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
words=sorted(set(words))
classes=sorted(set(classes))

In [5]:
training =[]
out_empty =[0]*len(classes)

for idx, doc in enumerate(data_x):
    bow=[]
    text=lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
        
    output_row=list(out_empty)
    output_row[classes.index(data_y[idx])]=1
    
    training.append([bow,output_row])
    
random.shuffle(training)
training=np.array(training,dtype=object)
train_x=np.array(list(training[:,0]))
train_y=np.array(list(training[:,1]))

In [6]:
model=Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),),activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation="softmax"))
adam=tf.keras.optimizers.legacy.Adam(learning_rate=0.01,decay=1e-6)
model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=["accuracy"])
print(model.summary())
model.fit(x=train_x,y=train_y,epochs=150,verbose=1)




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               14208     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 15)                975       
                                                                 
Total params: 23439 (91.56 KB)
Trainable params: 23439 (91.56 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/150




In [7]:
def clean_text(text):
    tokens = nltk.word_tokenize(text)
    tokens =[lemmatizer.lemmatize(word) for word in tokens]
    return tokens

def bag_of_words(text,vocab):
    tokens=clean_text(text)
    bow=[0]*len(vocab)
    for w in tokens:
        for idx, word in enumerate(vocab):
            if word ==w:
                bow[idx]=1
    return np.array(bow)

def pred_class(text,vocab,labels):
    bow=bag_of_words(text,vocab)
    result = model.predict(np.array([bow]))[0]
    thresh =0.5
    y_pred=[[indx,res] for indx, res in enumerate(result) if res > thresh]
    y_pred.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in y_pred:
        return_list.append(labels[r[0]])
    return return_list

def get_response(intents_list, intents_json):
    if len(intents_list)==0:
        result ="Sorry! I don't understand."
    else:
        tag=intents_list[0]
        list_of_intents=intents_json["intents"]
        for i in list_of_intents:
            if i["tag"]==tag:
                result = random.choice(i["responses"])
                break
    return result 



In [8]:
model.save("chatbot_model.h5")


c:\Users\kshit\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [9]:
print("Press 0 if you don't want to chat with our ChatBot.")
while True: 
    message = input("")
    if message == "0":
        break
    intents = pred_class(message,words,classes)
    result = get_response(intents,data)
    print(result)
    

Press 0 if you don't want to chat with our ChatBot.
1/1 [==============================] - 0s 71ms/step
Hello! I'm Dexter. How may I help you?
